In [1]:
import pdfminer
import requests
from bs4 import BeautifulSoup
from pdfminer import pdfparser
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import string
from scipy.spatial.distance import cosine
import numpy as np

In [2]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,\
                                  password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [3]:
x = convert_pdf_to_txt('/Users/Colin/Downloads/mathpdf.pdf')

In [4]:
type(x)

str

In [5]:
x = x.lower()

In [6]:
url = 'http://arxiv.org/list/math.GR/recent'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [7]:
text = r.text

In [8]:
pdfs = soup.findAll(title = 'Download PDF')

In [9]:
hrefs = soup.findAll(href = "/pdf/*")

In [10]:
link = str(pdfs[0]).split()[1].strip('href="')

In [11]:
links = [str(pdf).split()[1].strip('href="') for pdf in pdfs]

In [12]:
urls = ['http://arxiv.org'+ link for link in links]

In [13]:
type(urls[0])

str

In [14]:
soups = []
for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    soups.append(soup)

In [15]:
y = requests.get('http://arxiv.org/pdf/1508.00039.pdf')

In [16]:
import urllib2
from urllib2 import Request
from StringIO import StringIO

url = urls[0]

open = urllib2.urlopen(Request(url)).read()
memoryFile = StringIO(open)

#x = convert_pdf_to_txt(memoryFile)
#parser = pdfparser(memoryFile)

In [17]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import  TextConverter # , XMLConverter, HTMLConverter
import urllib2
from urllib2 import Request
import datetime
import re

# Define a PDF parser function
def parsePDF(url):

    # Open the url provided as an argument to the function and read the content
    open = urllib2.urlopen(Request(url)).read()

    # Cast to StringIO object
    from StringIO import StringIO
    memory_file = StringIO(open)

    # Create a PDF parser object associated with the StringIO object
    parser = PDFParser(memory_file)

    # Create a PDF document object that stores the document structure
    document = PDFDocument(parser)

    # Define parameters to the PDF device objet 
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    codec = 'utf-8'

    # Create a PDF device object
    device = TextConverter(rsrcmgr, retstr, codec = codec, laparams = laparams)

    # Create a PDF interpreter object
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    # Process each page contained in the document
    for page in PDFPage.create_pages(document):
        interpreter.process_page(page)
        data =  retstr.getvalue()
    return data

In [18]:
article = parsePDF(urls[0])

In [53]:
#steph = convert_pdf_to_txt('/Users/Colin/Downloads/plms.pdf')
#physics1 = convert_pdf_to_txt('/Users/Colin/Downloads/physics1.pdf')
shake = convert_pdf_to_txt('/Users/Colin/Downloads/shakespeare-tempest.pdf')

In [20]:
%time 
articles = []
for url in urls:
    articles.append(parsePDF(url))

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 5.96 µs


In [21]:
def get_text(url):
    base_url = url
    r = requests.get(base_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    pdfs = soup.findAll(title = 'Download PDF')
    links = [str(pdf).split()[1].strip('href="') for pdf in pdfs]
    urls = ['http://arxiv.org'+ link for link in links]
    articles = []
    for url in urls:
        articles.append(parsePDF(url))
    return articles

In [99]:
text = get_text('http://arxiv.org/list/math.NT/1508?show=150')

In [100]:
text.append(steph)
#text.append(physics1)
#text.append(shake)

In [101]:
S = set()
S.update(letter for letter in string.lowercase)
S.update(letter for letter in string.uppercase)
S.update(digit for digit in string.digits)

In [102]:
new_text=[]
for i in range(len(text)):
    new_text.append([word for word in text[i].split() if (word[0] in S) and (word[-1] in S)\
                    and (len(word)>3)])



In [103]:
for i in range(len(new_text)):
    new_text[i] = ' '.join(new_text[i])

In [104]:
Stop= set()
Stop.update([word for word in tfidf_math.get_stop_words()])
Stop.update(['theorem', 'lemma', 'proof', 'sum', 'difference', \
             'product', 'multiple', 'let', 'group', 'prime', 'log', 'limit', 'cid', 'result'\
            'main', 'conjecture'])
Stop = list(Stop)
'be' in Stop

True

In [105]:

tfidf_math = TfidfVectorizer(max_features = 20000, stop_words = Stop, ngram_range=(1,2))

In [106]:
M = tfidf_math.fit_transform(new_text)

In [107]:
M.shape

(103, 20000)

In [108]:
feature_names = tfidf_math.get_feature_names()

In [109]:
nmf = NMF(n_components=10)
nmf.fit(M)
for topic_idx, topic in enumerate(nmf.components_):
    print("Topic #%d:" % topic_idx)
    print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-10 - 1:-1]]))
    print '\n'

Topic #0:
galois abelian subgroup curve groups image proposition extensions element point


Topic #1:
fibonacci fn lucas 2n fibonacci lucas bicomplex fibonacci numbers fn fn sequence numbers


Topic #2:
random probability length cycles j6k cycles length qd cycle polynomials aλ


Topic #3:
modular forms eisenstein modular forms siegel modular siegel weight eisenstein series maass ag


Topic #4:
yi height bound linear lower matrix inequality bounds polynomial number


Topic #5:
winning game badly approximable alice badly approximable decaying absolute winning schmidt approximable numbers


Topic #6:
eigenfunctions quantum small scale scatterers point scatterers scale scale equidistribution momentum scars eigenvalues


Topic #7:
partitions partition ehrhart modulo crank figure quadratic lattice congruences binary


Topic #8:
zeta dx zeta function 18 hurwitz p1 2n euler ep 18 ep


Topic #9:
bound xn number log2 symmf function sums multiplicative zeros xx




In [127]:
N = M.todense()
distances = []
for ix in range(N.shape[0]):
    distances.append(cosine(N[4], N[ix]))
print np.argsort(distances)[1:10]

[ 16  24  42 101  86   2  37  13  85]


In [128]:
print np.around(sorted(distances)[1:10],4)

[ 0.6965  0.869   0.8729  0.8782  0.8877  0.8895  0.8922  0.9018  0.9021]
